안녕하세요. '김상동구' 팀입니다.

public 18등 (0.87619) , private 12등 (0.88049) 을 기록하였습니다.

이번 대회의 베이스라인을 기반으로 colab에서 코드를 작성하였습니다.

https://dacon.io/competitions/official/235894/codeshare/4722?page=1&dtype=recent

## 1. 전처리

이번 대회의 데이터셋 anomaly data의 양이 정상 데이터보다 개수가 훨씬 더 적었기 때문에 data augmentation을 사용해야 한다고 생각했습니다.

augmentation 모듈 중 하나인 albumentations를 사용했는데, 다음 링크를 참조하여 사용했습니다.

https://gaussian37.github.io/dl-pytorch-albumentation/

## 2. 모델 선정

모델은 efficientnet, wide resnet, inception, vgg 모델을 사용해서 앙상블을 해봤을 때 결과가 가장 좋았던 efficientnet_b3와 wide_resnet50_2의 조합을 사용했습니다.

gpu power가 제한적이어서 많은 시도를 못해봤던 것이 아쉬웠습니다.

## 3. 훈련

colab을 통해 코드를 돌리는 동안 런타임이 자주 끊기는 일이 발생하여서 매 epoch마다 모델을 저장하면서 training을 진행하였습니다.

시간과 computing power가 부족하여 efficientnet_b3 모델은 115 epoch, 
wide_resnet50_2 모델은 78 epoch까지 훈련했습니다.

## 4. 추론

데이터의 크기가 크고 훈련할 anomaly data의 양이 적고 시간이 많이 걸려서 따로 validation set을 만들지 않고, 제출을 해서 결과를 확인한 후 parameter를 조정하였습니다.

모델의 성능을 최대화하기 위해서 tta모듈 (test time augmentation)을 사용하여 추론하였습니다.

# Import Library

In [ ]:
!pip install timm

     |████████████████████████████████| 431 kB 5.3 MB/s 


In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62

In [ ]:
!pip install opencv-python-headless==4.5.2.52

     |████████████████████████████████| 38.2 MB 1.3 MB/s 


In [ ]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 6.7 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
!pip install ttach

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2
import gc

import torchvision
from torchvision import models

import timm
import os
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score, accuracy_score
import time

import albumentations as A
from albumentations.pytorch import ToTensorV2

import ttach as tta

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def seed_all(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)

In [ ]:
os.chdir("/content/drive/MyDrive/Dacon/Dacon_CV")

# Dataset 불러오기

In [ ]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [ ]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384))

    gc.collect()
    torch.cuda.empty_cache()
    
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [18:39<00:00,  1.92it/s]


# albumentations을 이용한 Data Augmentation

In [ ]:
train_transform = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
        A.RandomGamma(gamma_limit=(90, 110)),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10),
        A.Transpose(),
        A.RandomRotate90(),
        A.OneOf([A.NoOp(), A.MultiplicativeNoise(), A.GaussNoise(), A.ISONoise()]),
        A.OneOf(
            [
                A.NoOp(p=0.8),
                A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10),
                A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10)
            ],
            p=0.2,
        ),
        A.OneOf([A.ElasticTransform(), A.GridDistortion(), A.NoOp()]),
        ToTensorV2(),
])


test_transform = A.Compose([
        ToTensorV2()
])

# Configs

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            img = train_transform(image=img)
        
        if self.mode=='test':
            img = test_transform(image=img)
        
        label = self.labels[idx]
        return img, label


class Network_b3(nn.Module):
    def __init__(self):
        super(Network_b3, self).__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)

        return x

class Network_wrn(nn.Module):
    def __init__(self):
        super(Network_wrn, self).__init__()
        self.model = timm.create_model('wide_resnet50_2', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)

        return x

In [ ]:
batch_size = 32
epochs = 120

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

## efficientnet_b3

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score


model_b3 = Network_b3().to(device)

optimizer = torch.optim.Adam(model_b3.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

### Train

In [ ]:
'''path = './weights/'
if not os.path.isdir(path):
    os.mkdir(path)

for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model_b3.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0]['image'], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model_b3(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()

    train_f1 = score_function(train_y, train_pred)
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

    # 모델 저장
    torch.save({
                'epoch': epoch,
                'model_state_dict': model_b3.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                'loss': loss,
                }, f"{path}/b3_model.pt")'''

## wide_resnet50_2 model

In [ ]:
model_wrn = Network_wrn().to(device)

optimizer = torch.optim.Adam(model_wrn.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/wide_resnet50_racm-8234f177.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_racm-8234f177.pth


### Train

In [ ]:
'''path = './weights/'
if not os.path.isdir(path):
    os.mkdir(path)

for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model_wrn.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0]['image'], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model_wrn(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()

    train_f1 = score_function(train_y, train_pred)
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

    # 모델 저장
    torch.save({
                'epoch': epoch,
                'model_state_dict': model_wrn.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                'loss': loss,
                }, f"{path}/wrn_model.pt")'''

In [ ]:
# efficientnet_b3 모델 불러오기

loaded_model = torch.load('/content/drive/MyDrive/Dacon/Dacon_CV/weights/model_b3.pt')
model_b3 = Network_b3().to(device)
model_b3.load_state_dict(loaded_model['model_state_dict'])

<All keys matched successfully>

In [ ]:
# wide_resnet50_2 모델 불러오기

loaded_model = torch.load('/content/drive/MyDrive/Dacon/Dacon_CV/weights/model_wrn.pt')
model_wrn = Network_wrn().to(device)
model_wrn.load_state_dict(loaded_model['model_state_dict'])

<All keys matched successfully>

## TTA (test time augmentation)

In [ ]:
tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 90, 180, 270]),
        tta.Multiply([0.9, 1])
    ]
)

tta_model_b3 = tta.ClassificationTTAWrapper(model_b3, tta_transforms)
tta_model_wrn = tta.ClassificationTTAWrapper(model_wrn, tta_transforms)

# Inference

In [ ]:
tta_model_b3.eval()
tta_model_wrn.eval()
f_pred = []

with torch.no_grad  ():
    for batch in (test_loader):
        x = torch.tensor(batch[0]['image'], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            # ensemble
            pred_b3 = tta_model_b3(x)
            pred_wrn = tta_model_wrn(x)
            pred = pred_b3 + pred_wrn
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

# Submit

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

submission = pd.read_csv("open/sample_submission.csv")
submission["label"] = f_result

submission

In [ ]:
submission.to_csv("ensemble_b0_b3_inc_res_ver5.csv", index = False)